<a href="https://colab.research.google.com/github/kumuds4/BCH/blob/master/Making_the_Most_of_your_Colab_Subscription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Making the Most of your Colab Subscription



## Faster GPUs

Users who have purchased one of Colab's paid plans have access to faster GPUs and more memory. You can upgrade your notebook's GPU settings in `Runtime > Change runtime type` in the menu to select from several accelerator options, subject to availability.

The free of charge version of Colab grants access to Nvidia's T4 GPUs subject to quota restrictions and availability.

You can see what GPU you've been assigned at any time by executing the following cell. If the execution result of running the code cell below is "Not connected to a GPU", you can change the runtime by going to `Runtime > Change runtime type` in the menu to enable a GPU accelerator, and then re-execute the code cell.


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In order to use a GPU with your notebook, select the `Runtime > Change runtime type` menu, and then set the hardware accelerator to the desired option.

## More memory

Users who have purchased one of Colab's paid plans have access to high-memory VMs when they are available. More powerful GPUs are always offered with high-memory VMs.



You can see how much memory you have available at any time by running the following code cell. If the execution result of running the code cell below is "Not using a high-RAM runtime", then you can enable a high-RAM runtime via `Runtime > Change runtime type` in the menu. Then select High-RAM in the Runtime shape toggle button. After, re-execute the code cell.


In [ ]:
import psutil

ram_gb = psutil.virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

## Longer runtimes

All Colab runtimes are reset after some period of time (which is faster if the runtime isn't executing code). Colab Pro and Pro+ users have access to longer runtimes than those who use Colab free of charge.

## Background execution

Colab Pro+ users have access to background execution, where notebooks will continue executing even after you've closed a browser tab. This is always enabled in Pro+ runtimes as long as you have compute units available.



## Relaxing resource limits in Colab Pro

Your resources are not unlimited in Colab. To make the most of Colab, avoid using resources when you don't need them. For example, only use a GPU when required and close Colab tabs when finished.



If you encounter limitations, you can relax those limitations by purchasing more compute units via Pay As You Go. Anyone can purchase compute units via [Pay As You Go](https://colab.research.google.com/signup); no subscription is required.

## Send us feedback!

If you have any feedback for us, please let us know. The best way to send feedback is by using the Help > 'Send feedback...' menu. If you encounter usage limits in Colab Pro consider subscribing to Pro+.

If you encounter errors or other issues with billing (payments) for Colab Pro, Pro+, or Pay As You Go, please email [colab-billing@google.com](mailto:colab-billing@google.com).

## More Resources

### Working with Notebooks in Colab
- [Overview of Colab](/notebooks/basic_features_overview.ipynb)
- [Guide to Markdown](/notebooks/markdown_guide.ipynb)
- [Importing libraries and installing dependencies](/notebooks/snippets/importing_libraries.ipynb)
- [Saving and loading notebooks in GitHub](https://colab.research.google.com/github/googlecolab/colabtools/blob/main/notebooks/colab-github-demo.ipynb)
- [Interactive forms](/notebooks/forms.ipynb)
- [Interactive widgets](/notebooks/widgets.ipynb)

<a name="working-with-data"></a>
### Working with Data
- [Loading data: Drive, Sheets, and Google Cloud Storage](/notebooks/io.ipynb)
- [Charts: visualizing data](/notebooks/charts.ipynb)
- [Getting started with BigQuery](/notebooks/bigquery.ipynb)

### Machine Learning Crash Course
These are a few of the notebooks from Google's online Machine Learning course. See the [full course website](https://developers.google.com/machine-learning/crash-course/) for more.
- [Intro to Pandas DataFrame](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/pandas_dataframe_ultraquick_tutorial.ipynb)
- [Linear regression with tf.keras using synthetic data](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/linear_regression_with_synthetic_data.ipynb)


<a name="using-accelerated-hardware"></a>
### Using Accelerated Hardware
- [TensorFlow with GPUs](/notebooks/gpu.ipynb)
- [TPUs in Colab](/notebooks/tpu.ipynb)

<a name="machine-learning-examples"></a>

## Machine Learning Examples

To see end-to-end examples of the interactive machine learning analyses that Colab makes possible, check out these tutorials using models from [TensorFlow Hub](https://tfhub.dev).

A few featured examples:

- [Retraining an Image Classifier](https://tensorflow.org/hub/tutorials/tf2_image_retraining): Build a Keras model on top of a pre-trained image classifier to distinguish flowers.
- [Text Classification](https://tensorflow.org/hub/tutorials/tf2_text_classification): Classify IMDB movie reviews as either *positive* or *negative*.
- [Style Transfer](https://tensorflow.org/hub/tutorials/tf2_arbitrary_image_stylization): Use deep learning to transfer style between images.
- [Multilingual Universal Sentence Encoder Q&A](https://tensorflow.org/hub/tutorials/retrieval_with_tf_hub_universal_encoder_qa): Use a machine learning model to answer questions from the SQuAD dataset.
- [Video Interpolation](https://tensorflow.org/hub/tutorials/tweening_conv3d): Predict what happened in a video between the first and the last frame.


In [ ]:
mport numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import logging
import pandas as pd
import traceback

# Configure logging
logging.basicConfig(level=logging.INFO)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Configuration parameters
BLOCK_LENGTH = 128
INFO_BITS = 64
LEARNING_RATE = 1e-3
EPOCHS = 50
BATCH_SIZE = 32
NUM_SAMPLES_TRAIN = 50000
NUM_TRIALS_PERF = 1500
SNR_RANGE_AWGN = np.linspace(0, 10, 21)
LIST_SIZES = [1, 8, 16]

#part 2

def compute_crc(data, polynomial):
    poly_len = len(polynomial)
    crc_length = poly_len - 1
    data_with_zeros = np.concatenate((data, np.zeros(crc_length, dtype=int)))
    remainder = np.copy(data_with_zeros)
    for i in range(len(remainder) - poly_len + 1):
        if remainder[i] == 1:
            remainder[i:i + poly_len] ^= polynomial
    return remainder[-crc_length:]

class PolarCodeGenerator:
    def __init__(self, N, K, channel_snr_db, crc_type='CRC-7'):
        self.N = N
        self.K = K
        self.R = K / N
        self.channel_snr_db = channel_snr_db
        self.crc_type = crc_type
        self.crc_polynomials = {'CRC-7': (np.array([1, 0, 0, 0, 1, 0, 0, 1], dtype=int), 7)}

        if crc_type in self.crc_polynomials:
            self._crc_polynomial = self.crc_polynomials[crc_type][0]
            self._crc_length = self.crc_polynomials[crc_type][1]
        else:
            self._crc_polynomial = None
            self._crc_length = 0

        self.K_crc = self.K + self._crc_length
        self.frozen_set, self.info_set = self._get_frozen_and_info_sets()

    def _get_frozen_and_info_sets(self):
        if self.N == 128:
            reliability_sequence = self._get_3gpp_reliability_sequence_128()
            info_channel_indices = sorted(reliability_sequence[-self.K_crc:])
            frozen_channel_indices = sorted(list(set(range(self.N)) - set(info_channel_indices)))
        else:
            raise NotImplementedError("Reliability sequence for N != 128 is not implemented.")
        return frozen_channel_indices, info_channel_indices

    def _get_3gpp_reliability_sequence_128(self):
        return [
            0, 1, 2, 4, 8, 16, 3, 5, 9, 6, 17, 10, 18, 32, 12, 33,
            20, 24, 34, 36, 40, 7, 11, 19, 21, 13, 22, 25, 26, 28,
            48, 35, 37, 38, 41, 42, 44, 56, 14, 15, 23, 27, 29, 30,
            31, 39, 43, 45, 46, 49, 50, 52, 57, 58, 60, 63, 47, 51,
            53, 54, 59, 61, 62, 65, 66, 67, 68, 70, 72, 73, 74, 75,
            76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100, 77,
            79, 81, 83, 85, 87, 89, 91, 93, 95, 97, 99, 101, 102, 103,
            104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115,
            116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127
        ]

    def generate_info_bits(self):
        return np.random.randint(2, size=self.K)

    def polar_encode(self, info_bits):
        info_bits_with_crc = self.append_crc(info_bits)
        if len(info_bits_with_crc) != len(self.info_set):
            raise ValueError("Length mismatch")
        u = np.zeros(self.N, dtype=int)
        u[list(self.info_set)] = info_bits_with_crc
        encoded = self._polar_transform(u)
        return encoded
def _polar_transform(self, u):
        if len(u) == 1:
            return u
        else:
            half_N = len(u) // 2
            x_upper = self._polar_transform(u[:half_N])
            x_lower = self._polar_transform(u[half_N:])
            return np.concatenate([(x_upper + x_lower) % 2, x_lower])

def append_crc(self, info_bits):
        if self.crc_type not in self.crc_polynomials:
            return info_bits
        polynomial, length = self.crc_polynomials[self.crc_type]
        data_for_crc = np.copy(info_bits)
        crc_bits = compute_crc(data_for_crc, polynomial)
        if len(crc_bits) != length:
            logging.warning(f"Calculated CRC length ({len(crc_bits)}) does not match expected length ({length})")
        return np.concatenate((info_bits, crc_bits))

 #part 3

def bpsk_modulate(bits):
    bits = np.array(bits, dtype=int)
    return 2 * bits - 1

class EnhancedChannelSimulator:
    def __init__(self, channel_type='AWGN'):
        self.channel_type = channel_type

    def simulate(self, signal, snr_db):
        snr_linear = 10 ** (snr_db / 10)
        noise_std = np.sqrt(1 / (2 * snr_linear))
        noise = noise_std * np.random.randn(*signal.shape)
        return signal + noise

def prepare_polar_dataset(polar_code_gen, num_samples, snr_db=5, channel_type='AWGN'):
    channel_simulator = EnhancedChannelSimulator(channel_type=channel_type)
    X, y = [], []

    for _ in range(num_samples):
        info_bits = polar_code_gen.generate_info_bits()
        encoded_signal = polar_code_gen.polar_encode(info_bits)
        modulated_signal = bpsk_modulate(encoded_signal)
        received_signal = channel_simulator.simulate(modulated_signal, snr_db)
        X.append(received_signal)
        y.append(info_bits)

    return np.array(X), np.array(y)

def save_dataset_to_csv(X, y, filename='dataset.csv'):
    data = np.hstack((X, y))
    columns = [f'received_{i}' for i in range(X.shape[1])] + [f'bit_{j}' for j in range(y.shape[1])]
    df = pd.DataFrame(data, columns=columns)
    df.to_csv(filename, index=False)
    logging.info(f"Dataset saved to {filename}")

 #part 4
class EnhancedRNNDecoder(nn.Module):
    def __init__(self, input_size, output_size, hidden_size=128, num_layers=2):
        super(EnhancedRNNDecoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x_reshaped = x.unsqueeze(1)
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)
        out, _ = self.rnn(x_reshaped, (h0, c0))
        out = self.fc(out[:, -1, :])
        return self.sigmoid(out)

class DecoderTrainer:
    def __init__(self, model, learning_rate):
        self.model = model
        self.criterion = nn.BCELoss()
        self.optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    def train(self, X_train, y_train, X_val=None, y_val=None, epochs=50, batch_size=32):
        dataset = torch.utils.data.TensorDataset(X_train, y_train)
        loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

        train_losses, val_losses = [], []

        for epoch in range(epochs):
            epoch_loss = 0
            self.model.train()

            for X_batch, y_batch in loader:
                X_batch = X_batch.view(-1, BLOCK_LENGTH)
                self.optimizer.zero_grad()
                outputs = self.model(X_batch)
                loss = self.criterion(outputs, y_batch)
                loss.backward()
                self.optimizer.step()
                epoch_loss += loss.item()

            train_loss = epoch_loss / len(loader)
            train_losses.append(train_loss)
            logging.info(f"Epoch {epoch+1}/{epochs}, Training Loss: {train_loss:.4f}")

            if X_val is not None and y_val is not None:
                self.model.eval()
                with torch.no_grad():
                    val_output = self.model(X_val.view(-1, BLOCK_LENGTH))
                    val_loss = self.criterion(val_output, y_val).item()
                    val_losses.append(val_loss)
                    logging.info(f"Epoch {epoch+1}/{epochs}, Validation Loss: {val_loss:.4f}")

        return train_losses, val_losses if X_val is not None else None

    def evaluate(self, X_test, y_test):
        self.model.eval()
        with torch.no_grad():
            outputs = self.model(X_test.view(-1, BLOCK_LENGTH))
            predicted = (outputs > 0.5).int()
            total = y_test.numel()
            bit_errors = torch.sum(predicted != y_test).item()
            block_errors = torch.sum(torch.any(predicted != y_test, dim=1)).item()
            ber = bit_errors / total
            bler = block_errors / X_test.size(0)

        return ber, bler

# Part 5
############################################################################


lass PolarCodeDecoder:
    def __init__(self, N, K, list_size, crc_poly=None):
        self.N = N
        self.K = K
        self.list_size = list_size
        self.frozen_set = self._get_frozen_set()
        self.info_set = sorted(list(set(range(N)) - set(self.frozen_set)))
        self._crc_polynomial = crc_poly[0] if crc_poly else None
        self._crc_length = crc_poly[1] if crc_poly else 0

    def _get_frozen_set(self):
        return set(range(self.K, self.N))

    def decode(self, received_llrs):
        active_path_indices = list(range(self.list_size))
        self.paths = [[] for _ in range(self.list_size)]
        self.path_metrics = [0.0] * self.list_size
        self.hard_decisions = [np.zeros(self.N, dtype=int) for _ in range(self.list_size)]
        self.llrs = [np.copy(received_llrs) for _ in range(self.list_size)]

        final_active_path_indices = self._recursive_decode(active_path_indices, 0, self.N)
        best_path_index = np.argmin(self.path_metrics)
        return self.hard_decisions[best_path_index][list(self.info_set)]

    def _recursive_decode(self, active_path_indices, bit_index, block_size):
        if block_size == 1:
            for path_idx in active_path_indices:
                llr = self.llrs[path_idx][bit_index]
                if bit_index in self.frozen_set:
                    self.hard_decisions[path_idx][bit_index] = 0
                else:
                    self.hard_decisions[path_idx][bit_index] = 0 if llr >= 0 else 1
            return active_path_indices
        else:
            half_size = block_size // 2
            for path_idx in active_path_indices:
                llr_f = self._f(self.llrs[path_idx][bit_index:bit_index + half_size],
                                self.llrs[path_idx][bit_index + half_size:bit_index + block_size])
                self.llrs[path_idx][bit_index:bit_index + half_size] = llr_f

            active_paths_after_u1 = self._recursive_decode(active_path_indices, bit_index, half_size)

            for path_idx in active_paths_after_u1:
                u1_decisions = self.hard_decisions[path_idx][bit_index:bit_index + half_size]
                llr_g = self._g(self.llrs[path_idx][bit_index:bit_index + half_size],
                                self.llrs[path_idx][bit_index + half_size:bit_index + block_size], u1_decisions)
                self.llrs[path_idx][bit_index + half_size:bit_index + block_size] = llr_g

            return self._recursive_decode(active_paths_after_u1, bit_index + half_size, half_size)

    def _f(self, L1, L2):
        return np.minimum(np.abs(L1), np.abs(L2)) * np.sign(L1) * np.sign(L2)

    def _g(self, L1, L2, u1):
        return L2 + (1 - 2 * u1) * L1

    def compute_crc(data, polynomial):
    """
    Computes CRC for data using the given polynomial.
    Args:
        data: Numpy array of binary data bits (0s and 1s).
        polynomial: Numpy array of binary polynomial coefficients (e.g., [1, 0, 0, 0, 1, 0, 0, 1] for x^7 + x^3 + 1).
    Returns:
        Numpy array of CRC bits.
    """
    poly_len = len(polynomial)
    crc_length = poly_len - 1
    data_with_zeros = np.concatenate((data, np.zeros(crc_length, dtype=int)))
    remainder = np.copy(data_with_zeros)

    for i in range(len(remainder) - poly_len + 1):
        if remainder[i] == 1:
            remainder[i:i + poly_len] ^= polynomial

    return remainder[-crc_length:]
#############################################################################
#part 6 Plotting fuctions

def plot_ber_bler_comparison(snr_range, rnn_results, scl_results, sc_results, list_sizes):
    plt.figure(figsize=(18, 6))

    # BER Plot
    plt.subplot(1, 2, 1)
    plt.yscale('log')
    plt.ylim(1e-4, 1)
    plt.plot(snr_range, rnn_results['BER_RNN'], label='RNN')
    for size in list_sizes:
        plt.plot(snr_range, [result['BER'] for result in scl_results[size]], label=f'SCL, List Size {size}')
    plt.plot(snr_range, sc_results['BER_SC'], label='SC')
    plt.xlabel('SNR (dB)')
    plt.ylabel('BER')
    plt.title('Bit Error Rate (BER)')
    plt.legend()
    plt.grid(True, which="both", ls="--")

    # BLER Plot
    plt.subplot(1, 2, 2)
    plt.yscale('log')
    plt.ylim(1e-4, 1)
    plt.plot(snr_range, rnn_results['BLER_RNN'], label='RNN')
    for size in list_sizes:
        plt.plot(snr_range, [result['BLER'] for result in scl_results[size]], label=f'SCL, List Size {size}')
    plt.plot(snr_range, sc_results['BLER_SC'], label='SC')
    plt.xlabel('SNR (dB)')
    plt.ylabel('BLER')
    plt.title('Block Error Rate (BLER)')
    plt.legend()
    plt.grid(True, which="both", ls="--")

    plt.tight_layout()
    plt.show()

def plot_ber_bler_single(snr_range, ber, bler, name=''):
    plt.figure(figsize=(10,5))

    plt.subplot(1,2,1)
    plt.yscale('log')
    plt.ylim(1e-4, 1)
    plt.plot(snr_range, ber, marker='o')
    plt.xlabel('SNR (dB)')
    plt.ylabel('BER')
    plt.title(f'{name} BER')
    plt.grid(True, which="both", ls="--")
    ax = plt.gca()
    ax.yaxis.set_major_formatter(LogFormatterMathtext())

    plt.subplot(1,2,2)
    plt.yscale('log')
    plt.ylim(1e-4, 1)
    plt.plot(snr_range, bler, marker='o')
    plt.xlabel('SNR (dB)')
    plt.ylabel('BLER')
    plt.title(f'{name} BLER')
    plt.grid(True, which="both", ls="--")
    ax = plt.gca()
    ax.yaxis.set_major_formatter(LogFormatterMathtext())

    plt.tight_layout()
    plt.show()

def plot_ber_bler_single(snr_range, ber, bler, name=''):
    plt.figure(figsize=(10,5))
    plt.subplot(1,2,1)
    plt.yscale('log')
    plt.ylim(1e-4, 1)
    plt.plot(snr_range, ber, marker='o')
    plt.xlabel('SNR (dB)')
    plt.ylabel('BER')
    plt.title(f'{name} BER')
    plt.grid(True, which="both", ls="--")
    plt.subplot(1,2,2)
    plt.yscale('log')
    plt.ylim(1e-4, 1)
    plt.plot(snr_range, bler, marker='o')
    plt.xlabel('SNR (dB)')
    plt.ylabel('BLER')
    plt.title(f'{name} BLER')
    plt.grid(True, which="both", ls="--")
    plt.tight_layout()
    plt.show()

# For RNN:
plot_ber_bler_single(SNR_RANGE_AWGN, rnn_results['BER_RNN'], rnn_results['BLER_RNN'], name='RNN')

# For SC:
plot_ber_bler_single(SNR_RANGE_AWGN, sc_results['BER_SC'], sc_results['BLER_SC'], name='SC')

# For SCL (L=8, for example):
plot_ber_bler_single(
    SNR_RANGE_AWGN,
    [d['BER'] for d in scl_results[8]],
    [d['BLER'] for d in scl_results[8]],
    name='SCL L=8'
)

######################################################################
def plot_training_validation(train_losses, val_losses):
    plt.figure(figsize=(8, 4))
    plt.plot(train_losses, label='Training Loss')
    if val_losses:
        plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.show()

def plot_ber_bler_comparison(snr_range, rnn_results, scl_results, list_sizes):
    plt.figure(figsize=(12, 6))

    # BER Plot
    plt.subplot(1, 2, 1)
    plt.yscale('log')
    plt.ylim(1e-4, 1)
    plt.plot(snr_range, rnn_results['BER_RNN'], label='RNN')
    for size in list_sizes:
        plt.plot(snr_range, [result['BER'] for result in scl_results[size]], label=f'SCL, List Size {size}')
    plt.xlabel('SNR (dB)')
    plt.ylabel('BER')
    plt.title('Bit Error Rate (BER)')
    plt.legend()
    plt.grid(True, which="both", ls="--")

    # BLER Plot
    plt.subplot(1, 2, 2)
    plt.yscale('log')
    plt.ylim(1e-4, 1)
    plt.plot(snr_range, rnn_results['BLER_RNN'], label='RNN')
    for size in list_sizes:
        plt.plot(snr_range, [result['BLER'] for result in scl_results[size]], label=f'SCL, List Size {size}')
    plt.xlabel('SNR (dB)')
    plt.ylabel('BLER')
    plt.title('Block Error Rate (BLER)')
    plt.legend()
    plt.grid(True, which="both", ls="--")

    plt.tight_layout()
    plt.show()

def plot_confusion_matrix(y_true, y_pred, title='Confusion Matrix'):
    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()
    plt.title(title)
    plt.show()

#Part 7: Main Function


def main():
    try:
        # Set up the device
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Configuration parameters
        BLOCK_LENGTH = 128
        INFO_BITS = 64
        LEARNING_RATE = 1e-3
        EPOCHS = 50
        BATCH_SIZE = 32
        NUM_SAMPLES_TRAIN = 50000
        NUM_TRIALS_PERF = 1500
        SNR_RANGE_AWGN = np.linspace(0, 10, 21)
        LIST_SIZES = [1, 8, 16]

        # Polar code generator initialization
        polar_code_gen = PolarCodeGenerator(N=BLOCK_LENGTH, K=INFO_BITS)
        rnn_model = EnhancedRNNDecoder(BLOCK_LENGTH, INFO_BITS).to(device)
        rnn_trainer = DecoderTrainer(rnn_model, LEARNING_RATE)

        # Generate and save dataset
        X_raw, y_raw = prepare_polar_dataset(
            polar_code_gen, num_samples=NUM_SAMPLES_TRAIN, snr_db=5.0, channel_type='AWGN'
        )
        save_dataset_to_csv(X_raw, y_raw, 'awgn_dataset.csv')

        # Reshape and split data, moving to the device
        X_tensor = torch.FloatTensor(X_raw).view(-1, BLOCK_LENGTH).to(device)
        y_tensor = torch.FloatTensor(y_raw).view(-1, INFO_BITS).to(device)

        # Split data
        train_size = int(0.8 * X_tensor.shape[0])
        train_X = X_tensor[:train_size]
        train_y = y_tensor[:train_size]
        val_X = X_tensor[train_size:]
        val_y = y_tensor[train_size:]

        # RNN Model Training
        train_losses, val_losses = rnn_trainer.train(
            train_X, train_y, X_val=val_X, y_val=val_y, epochs=EPOCHS, batch_size=BATCH_SIZE
        )

        # Performance comparison
        rnn_perf_results, scl_results = performance_comparison(
            rnn_trainer, polar_code_gen, SNR_RANGE_AWGN, 'AWGN', LIST_SIZES, NUM_TRIALS_PERF
        )

        # Plot results
        plot_training_validation(train_losses, val_losses)
        plot_ber_bler_comparison(SNR_RANGE_AWGN, rnn_perf_results, scl_results, LIST_SIZES)

        # Example Confusion Matrix
        y_true_example = train_y[:100].cpu().numpy()
        rnn_input_example = train_X[:100]
        rnn_output_prob_example = rnn_trainer.model(rnn_input_example).cpu().detach().numpy()
        rnn_output_example = (rnn_output_prob_example > 0.5).astype(int)
        y_pred_example = rnn_output_example.squeeze()
        plot_confusion_matrix(y_true_example.flatten(), y_pred_example.flatten(), title='Confusion Matrix')

            # RNN plot
        plot_ber_bler_single(SNR_RANGE_AWGN, rnn_results['BER_RNN'], rnn_results['BLER_RNN'], name='RNN')

        # SC plot
        plot_ber_bler_single(SNR_RANGE_AWGN, sc_results['BER_SC'], sc_results['BLER_SC'], name='SC')

        # SCL plot (for each list size separately)
        for L in LIST_SIZES:
            plot_ber_bler_single(
                SNR_RANGE_AWGN,
                [d['BER'] for d in scl_results[L]],
                [d['BLER'] for d in scl_results[L]],
                name=f'SCL L={L}'
            )


    except Exception as e:
        logging.error(f"Simulation Error: {e}")
        traceback.print_exc()

if __name__ == "__main__":
    main()

